In [ ]:
from modelforge.utils.profiling import measure_performance_for_edge_sizes, plot_computation_time, plot_gpu_memory_usage

In [ ]:
# Example usage:
edge_sizes = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5]  # Edge sizes in nanometers
potential_names = ['painn', 'schnet', 'physnet', 'ani2x', 'aimnet2', 'sake'] 

results = measure_performance_for_edge_sizes(
    edge_sizes=edge_sizes,
    potential_names=potential_names,
)

# Print the results
for result in results:
    print(f"Potential: {result['potential_name']}, "
          f"Edge Size: {result['edge_size_nm']} nm, "
          f"Number of Waters: {result['num_waters']}, "
          f"Memory Usage: {result['memory_usage_bytes']/1e6:.2f} MB, "
          f"Computation Time: {result['computation_time_s']*1000:.2f} ms")

# Plot the computation time
plot_computation_time(results)
plot_gpu_memory_usage(results)